In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import keras_tuner as kt
from sklearn.cluster import KMeans

In [ ]:
dataset_path = "path_to_intel_dataset" 

In [ ]:
img = cv2.imread(r"D:\Download\archive\seg_test\seg_test\buildings\20061.jpg")
print(np.min(img), np.max(img))

In [ ]:
img_size = (128, 128)
batch_size = 32

In [ ]:
def load_images(dataset_path, img_size):
    images = []
    for img_name in os.listdir(dataset_path):
        img_path = os.path.join(dataset_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size) / 255.0
        images.append(img)
    
    return np.array(images)

In [ ]:
X = load_images(dataset_path, img_size)
print("Dataset shape:", X.shape)

In [ ]:
def plot_sample_images(images, num_samples=10):
    plt.figure(figsize=(12, 5))
    
    for i in range(num_samples):
        idx = np.random.randint(0, len(images))
        plt.subplot(2, num_samples//2, i+1)
        plt.imshow(images[idx])
        plt.axis("off")
    
    plt.suptitle("Sample Images from Dataset")
    plt.show()

plot_sample_images(X, num_samples=10)

In [ ]:
def plot_pixel_distribution(images, num_samples=100):
    sampled_images = images[np.random.choice(len(images), num_samples, replace=False)]
    pixel_values = sampled_images.flatten() 
    
    plt.figure(figsize=(8, 5))
    plt.hist(pixel_values, bins=50, color="blue", alpha=0.7)
    plt.title("Pixel Intensity Distribution")
    plt.xlabel("Pixel Value (0-1)")
    plt.ylabel("Frequency")
    plt.show()

plot_pixel_distribution(X, num_samples=100)

In [ ]:
def plot_rgb_histogram(image):
    channels = ["Red", "Green", "Blue"]
    colors = ["r", "g", "b"]
    
    plt.figure(figsize=(8, 5))
    
    for i, (channel, color) in enumerate(zip(channels, colors)):
        hist_values = image[:, :, i].flatten()
        plt.hist(hist_values, bins=50, color=color, alpha=0.5, label=channel)
    
    plt.title("RGB Color Distribution")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()

random_idx = np.random.randint(0, len(X))
plot_rgb_histogram(X[random_idx])

In [ ]:
def build_autoencoder(hp):
    latent_dim = hp.Choice('latent_dim', [32, 64, 128])

    inputs = keras.Input(shape=(128, 128, 3))
    x = layers.Conv2D(hp.Int("filters_1", 32, 64, step=16), (3, 3), activation="relu", padding="same")(inputs)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)
    x = layers.Conv2D(hp.Int("filters_2", 64, 128, step=32), (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)
    x = layers.Flatten()(x)
    latent = layers.Dense(latent_dim, activation="relu")(x)

    x = layers.Dense(32 * 32 * latent_dim, activation="relu")(latent)
    x = layers.Reshape((32, 32, latent_dim))(x)
    x = layers.Conv2DTranspose(hp.Int("filters_2", 64, 128, step=32), (3, 3), activation="relu", padding="same")(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(hp.Int("filters_1", 32, 64, step=16), (3, 3), activation="relu", padding="same")(x)
    x = layers.UpSampling2D((2, 2))(x)
    outputs = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

    autoencoder = keras.Model(inputs, outputs)
    autoencoder.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])),
        loss="mse"
    )

    return autoencoder

In [ ]:
tuner = kt.RandomSearch(
    build_autoencoder,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=1,
    directory="autoencoder_tuning",
    project_name="intel_autoencoder"
)

In [ ]:
tuner.search(X, X, epochs=10, batch_size=16, validation_split=0.2)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Best Hyperparameters:
- Latent Dim: {best_hps.get('latent_dim')}
- Filters 1: {best_hps.get('filters_1')}
- Filters 2: {best_hps.get('filters_2')}
- Learning Rate: {best_hps.get('learning_rate')}
""")

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [ ]:
idx = np.random.randint(0, len(X))
original = X[idx]
reconstructed = best_model.predict(np.expand_dims(original, axis=0))[0]

plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plt.imshow(original)
plt.title("Original")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(reconstructed)
plt.title("Reconstructed")
plt.axis("off")

plt.show()

In [ ]:
encoder = keras.Model(inputs, latent)

In [ ]:
X_latent = encoder.predict(X)

In [ ]:
num_clusters = 5

In [ ]:
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(X_latent)

In [ ]:
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(X[i])
    plt.axis("off")
    plt.title(f"Cluster {clusters[i]}")

plt.show()